# Primitives: Figures and Animations

Visual diagnostics for core primitives: Sobol Brownian vs Gaussian, and
bisection root convergence.

In [5]:
# Visual style for figures (accessible color cycle, high-DPI)
try:
    from bsde_dsgE.utils.nb_style import apply_notebook_style
    apply_notebook_style()
except Exception as e:
    print('Style setup skipped:', e)


Style setup skipped: 'grid.major.color is not a valid rc parameter (see rcParams.keys() for a list of valid parameters)'


In [6]:
# Ensure project root is on sys.path so `bsde_dsgE` (one level up) can be imported
import sys
from pathlib import Path

pkg_name = "bsde_dsgE"
nb_cwd = Path.cwd().resolve()

# Walk up to find the folder that directly contains the package directory
repo_root = None
for p in [nb_cwd, *nb_cwd.parents]:
    if (p / pkg_name).is_dir():
        repo_root = p
        break

if repo_root is not None and str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Verify import and show where it's coming from
try:
    import bsde_dsgE  # noqa: F401
    print(f"bsde_dsgE import OK from: {(Path(bsde_dsgE.__file__).parent)}")
except Exception as e:
    print("Could not import bsde_dsgE by path bootstrap. If this persists, install the project in editable mode from the repo root:")
    print("pip install -e .")
    raise

bsde_dsgE import OK from: C:\Users\fababa\Dropbox\AssetPricing\DSGE_BSDE\DSGE_BSDE\bsde_dsgE


## Rolling Increments Histogram (Animation)

Animate the histogram of Brownian increments over time steps for Sobol vs Gaussian to visualise distribution stability.

In [7]:
from matplotlib import animation
figA, (axS, axG) = plt.subplots(1,2, figsize=(8,3), sharey=True)
axS.set_title('Sobol increments'); axG.set_title('Gaussian increments')
bins = np.linspace(-3*np.sqrt(dt), 3*np.sqrt(dt), 30)
def init_hist():
    axS.cla(); axG.cla()
    return []
def animate_hist(i):
    axS.cla(); axG.cla()
    axS.hist(np.array(sob[:, i, 0]), bins=bins, alpha=0.8, color='tab:blue')
    axG.hist(np.array(gauss[:, i]), bins=bins, alpha=0.8, color='tab:orange')
    axS.set_ylim(0, max(axS.get_ylim()[1], axG.get_ylim()[1]))
    axS.set_title(f'Sobol increments step {i+1}')
    axG.set_title(f'Gaussian increments step {i+1}')
    return []
ani_hist = animation.FuncAnimation(figA, animate_hist, init_func=init_hist, frames=min(steps, 50), interval=150, blit=False)
ani_hist.to_jshtml()


NameError: name 'plt' is not defined

In [ ]:
import numpy as np
import jax, jax.numpy as jnp
import matplotlib.pyplot as plt
from bsde_dsgE.utils.sde_tools import sobol_brownian
from bsde_dsgE.core.outer_loop import pareto_bisection
FAST = bool(__import__('os').environ.get('NOTEBOOK_FAST', ''))


## Sobol Brownian vs Gaussian

In [ ]:
steps = 64 if FAST else 256
batch = 32 if FAST else 256
dt = 1.0 / steps
sob = sobol_brownian(dim=1, steps=steps, batch=batch, dt=dt)[...,0]
gauss = jax.random.normal(jax.random.PRNGKey(0), (batch, steps)) * np.sqrt(dt)
sob_path = np.cumsum(np.array(sob[0]))
gauss_path = np.cumsum(np.array(gauss[0]))
fig, ax = plt.subplots(1,3, figsize=(14,3))
ax[0].plot(sob_path); ax[0].set_title('Sobol Brownian (1 path)')
ax[1].plot(gauss_path); ax[1].set_title('Gaussian Brownian (1 path)')
# Histogram of increments comparison
ax[2].hist(np.array(sob).ravel(), bins=30, alpha=0.6, label='Sobol')
ax[2].hist(np.array(gauss).ravel(), bins=30, alpha=0.6, label='Gaussian')
ax[2].legend(); ax[2].set_title('Increments histogram')
fig.tight_layout(); fig


## Bisection Convergence Animation

In [ ]:
import matplotlib.animation as animation

def f(a): return a - 0.3  # root at 0.3
lo, hi = 0.0, 1.0
frames = []
for _ in range(20):
    mid = 0.5*(lo+hi)
    frames.append((lo, hi, mid))
    if f(mid) > 0: hi = mid
    else: lo = mid

fig2, ax2 = plt.subplots(figsize=(5,3))
ax2.set_xlim(0,1); ax2.set_ylim(-0.1, 1.1)
line, = ax2.plot([], [], 'k-')
lo_pt, = ax2.plot([], [], 'ro'); hi_pt, = ax2.plot([], [], 'bo'); mid_pt, = ax2.plot([], [], 'go')
ax2.plot([0,1], [f(0), f(1)], alpha=0.0)  # fix axes

def init():
    line.set_data([], [])
    lo_pt.set_data([], [])
    hi_pt.set_data([], [])
    mid_pt.set_data([], [])
    return line, lo_pt, hi_pt, mid_pt

def animate(i):
    lo, hi, mid = frames[i]
    xs = np.linspace(lo, hi, 100)
    ys = xs - 0.3
    line.set_data(xs, ys)
    lo_pt.set_data([lo], [f(lo)])
    hi_pt.set_data([hi], [f(hi)])
    mid_pt.set_data([mid], [f(mid)])
    return line, lo_pt, hi_pt, mid_pt

ani = animation.FuncAnimation(fig2, animate, init_func=init, frames=len(frames), interval=200, blit=True)
ani.to_jshtml()


## Variance growth and antithetic pairing

We confirm that var(B_t) ≈ t and show the effect of antithetic pairing (Sobol generator uses pairing).

In [ ]:
times = np.arange(1, steps+1) * dt
sob_paths = np.cumsum(np.array(sob), axis=1)
gauss_paths = np.cumsum(np.array(gauss), axis=1)
sob_var = sob_paths.var(axis=0)
gauss_var = gauss_paths.var(axis=0)
fig3, ax3 = plt.subplots(figsize=(6,3))
ax3.plot(times, sob_var, label='Sobol var')
ax3.plot(times, gauss_var, label='Gaussian var')
ax3.plot(times, times, 'k--', alpha=0.5, label='t')
ax3.legend(); ax3.set_title('Variance growth vs time'); ax3.set_xlabel('t'); ax3.set_ylabel('var')
fig3.tight_layout(); fig3


## ResNetND Feature Maps

Visualise outputs of a small `ResNetND` over a 2D input grid. We plot the scalar head `y(x)` and the two `z` components.

In [ ]:
import numpy as np
import jax, jax.numpy as jnp
from bsde_dsgE.core.nets import ResNetND
net2 = ResNetND.make(dim=2, depth=2, width=16, key=jax.random.PRNGKey(0))
# Build a grid in R^2
g = np.linspace(-1.5, 1.5, 60)
X1, X2 = np.meshgrid(g, g)
grid = jnp.stack([jnp.array(X1).ravel(), jnp.array(X2).ravel()], axis=1)
y, z = net2(jnp.zeros((grid.shape[0],)), grid)
Y = np.array(y).reshape(X1.shape)
Z1 = np.array(z[:,0]).reshape(X1.shape)
Z2 = np.array(z[:,1]).reshape(X1.shape)
fig4, ax4 = plt.subplots(1,3, figsize=(12,3), constrained_layout=True)
# Feature maps: use perceptually uniform cmaps; diverging for signed fields
extent = [g.min(), g.max(), g.min(), g.max()]
im0 = ax4[0].imshow(Y, extent=extent, origin='lower', aspect='equal', cmap='viridis'); ax4[0].set_title('y(x)')
m1 = float(np.max(np.abs(Z1))) if Z1.size else 1.0
m2 = float(np.max(np.abs(Z2))) if Z2.size else 1.0
im1 = ax4[1].imshow(Z1, extent=extent, origin='lower', aspect='equal', cmap='coolwarm', vmin=-m1, vmax=m1); ax4[1].set_title('z1(x)')
im2 = ax4[2].imshow(Z2, extent=extent, origin='lower', aspect='equal', cmap='coolwarm', vmin=-m2, vmax=m2); ax4[2].set_title('z2(x)')
c0 = fig4.colorbar(im0, ax=ax4[0]); c0.set_label('y')
c1 = fig4.colorbar(im1, ax=ax4[1]); c1.set_label('z1')
c2 = fig4.colorbar(im2, ax=ax4[2]); c2.set_label('z2')
fig4


## Antithetic Pairing Demo (Sobol Brownian)

Sobol generator uses antithetic pairing. We verify path pairs are negatives (correlation ≈ −1) and visualise one pair.

In [ ]:
pair_a = np.array(sob[0]).ravel()
pair_b = np.array(sob[batch//2]).ravel()
corr = np.corrcoef(pair_a, pair_b)[0,1]
print({'pair_corr': float(corr)})
# Show one paired path vs its antithetic
pa = np.cumsum(np.array(sob[0,:,0]))
pb = np.cumsum(np.array(sob[batch//2,:,0]))
figP, axP = plt.subplots(figsize=(5,3))
axP.plot(pa, label='path A'); axP.plot(pb, label='path B (anti)')
axP.legend(); axP.set_title('Antithetic path pair (Sobol)'); figP.tight_layout(); figP


## QQ Plot of Increments (Sobol vs Gaussian)

Compare empirical increment distributions to Normal(0, dt) via QQ plots.

In [ ]:
from bsde_dsgE.utils.figures import qq_points
qS, eS = qq_points(np.array(sob).ravel())
qG, eG = qq_points(np.array(gauss).ravel())
figQ, axQ = plt.subplots(1,2, figsize=(10,4), sharex=True, sharey=True)
axQ[0].plot(qS, eS, '.', alpha=0.5); axQ[0].plot(qS, qS, 'k--', lw=1)
axQ[0].set_title('Sobol increments QQ')
axQ[1].plot(qG, eG, '.', alpha=0.5); axQ[1].plot(qG, qG, 'k--', lw=1)
axQ[1].set_title('Gaussian increments QQ')
for ax in axQ: ax.set_xlabel('Theoretical'); ax.set_ylabel('Empirical'); ax.set_aspect('equal', adjustable='box')
figQ.tight_layout(); figQ


## Lagged Autocorrelation of Increments

Autocorrelation at lag 1 should be near 0 for independent increments (Gaussian and Sobol).

In [ ]:
from bsde_dsgE.utils.figures import lag_autocorr
sob_inc = np.array(sob)[..., None]  # (B, S) -> (B,S,1)
sob_inc = np.transpose(sob_inc, (1,0,2))  # (S,B,1)
gauss_inc = np.array(gauss)[..., None]  # (B,S)->(B,S,1)
gauss_inc = np.transpose(gauss_inc, (1,0,2))
ac_s = lag_autocorr(sob_inc, lag=1)[0]
ac_g = lag_autocorr(gauss_inc, lag=1)[0]
print({'lag1_autocorr_sobol': float(ac_s), 'lag1_autocorr_gaussian': float(ac_g)})
figAC, axAC = plt.subplots(figsize=(5,3))
labels = ['Sobol','Gaussian']; vals = [float(ac_s), float(ac_g)]
bars = axAC.bar(labels, vals, color=['tab:blue','tab:orange'])
axAC.axhline(0, color='k', lw=1); axAC.set_ylim(-0.2, 0.2)
for i, b in enumerate(bars):
    y = b.get_height(); axAC.annotate(f'{vals[i]:+.3f}',
        xy=(b.get_x()+b.get_width()/2, y), xytext=(0, 4), textcoords='offset points',
        ha='center', va='bottom', fontsize=9)
axAC.set_title('Lag-1 autocorrelation of increments'); figAC.tight_layout(); figAC
